In [1]:
"""
NOTEBOOK 5: COMPARAISON FINALE + ÉVALUATION TEST SET
====================================================
Ce notebook va:
1. Comparer tous les résultats des 3 datasets
2. Identifier le champion
3. TESTER sur le TEST SET pour vérifier l'overfitting
4. Donner le verdict final
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import warnings
warnings.filterwarnings('ignore')

# ============================================
# PARTIE 1: CHARGER TOUS LES RÉSULTATS
# ============================================
print("="*80)
print("📊 CHARGEMENT DES RÉSULTATS DE TOUS LES DATASETS")
print("="*80)

# Charger les 3 fichiers de résultats
results_symptoms = pd.read_csv('../results/metrics/symptoms_models_comparison.csv')
results_clinical = pd.read_csv('../results/metrics/clinical_models_comparison.csv')
results_pima = pd.read_csv('../results/metrics/pima_models_comparison.csv')

# Ajouter colonne dataset
results_symptoms['Dataset'] = 'Symptoms (600)'
results_clinical['Dataset'] = 'Clinical (10000)'
results_pima['Dataset'] = 'Pima (768)'

# Combiner tous les résultats
all_results = pd.concat([results_symptoms, results_clinical, results_pima], ignore_index=True)

print(f"\n✅ Total: {len(all_results)} modèles entraînés")
print(f"   - Symptoms: {len(results_symptoms)} modèles")
print(f"   - Clinical: {len(results_clinical)} modèles")
print(f"   - Pima: {len(results_pima)} modèles")

# ============================================
# PARTIE 2: TOP MODÈLES PAR DATASET
# ============================================
print("\n" + "="*80)
print("🏆 MEILLEURS MODÈLES PAR DATASET (selon F1-Score)")
print("="*80)

for dataset_name in ['Symptoms (600)', 'Clinical (10000)', 'Pima (768)']:
    subset = all_results[all_results['Dataset'] == dataset_name].sort_values('F1-Score', ascending=False)
    best = subset.iloc[0]
    
    print(f"\n📁 {dataset_name}:")
    print(f"   🥇 Modèle: {best['Model']}")
    print(f"   Accuracy:  {best['Accuracy']:.4f}")
    print(f"   Precision: {best['Precision']:.4f}")
    print(f"   Recall:    {best['Recall']:.4f}")
    print(f"   F1-Score:  {best['F1-Score']:.4f}")
    print(f"   ROC-AUC:   {best['ROC-AUC']:.4f}")

# ============================================
# PARTIE 3: CHAMPION ABSOLU (VALIDATION SET)
# ============================================
print("\n" + "="*80)
print("🏆 CHAMPION ABSOLU (sur Validation Set)")
print("="*80)

best_overall = all_results.sort_values('F1-Score', ascending=False).iloc[0]
print(f"\n🥇 Modèle: {best_overall['Model']}")
print(f"📁 Dataset: {best_overall['Dataset']}")
print(f"📊 Métriques:")
print(f"   Accuracy:  {best_overall['Accuracy']:.4f}")
print(f"   Precision: {best_overall['Precision']:.4f}")
print(f"   Recall:    {best_overall['Recall']:.4f}")
print(f"   F1-Score:  {best_overall['F1-Score']:.4f}")
print(f"   ROC-AUC:   {best_overall['ROC-AUC']:.4f}")

print("\n⚠️  ATTENTION: 100% sur validation peut indiquer de l'overfitting!")
print("   → On va TESTER sur le TEST SET pour vérifier...")

# ============================================
# PARTIE 4: VISUALISATION COMPARATIVE
# ============================================
print("\n" + "="*80)
print("📈 CRÉATION DES VISUALISATIONS COMPARATIVES")
print("="*80)

# Heatmap F1-Score
pivot = all_results.pivot_table(
    values='F1-Score', 
    index='Model', 
    columns='Dataset'
)

plt.figure(figsize=(12, 10))
sns.heatmap(pivot, annot=True, fmt='.3f', cmap='RdYlGn', 
            vmin=0.5, vmax=1.0, cbar_kws={'label': 'F1-Score'})
plt.title('Heatmap F1-Score: Tous Modèles sur Tous Datasets', fontsize=16, fontweight='bold')
plt.xlabel('Dataset', fontsize=12, fontweight='bold')
plt.ylabel('Modèle', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig('../results/visualizations/heatmap_all_datasets.png', dpi=300, bbox_inches='tight')
print("✅ Sauvegardé: heatmap_all_datasets.png")
plt.close()

# Graphique comparatif par dataset
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

for idx, dataset_name in enumerate(['Symptoms (600)', 'Clinical (10000)', 'Pima (768)']):
    subset = all_results[all_results['Dataset'] == dataset_name].sort_values('F1-Score', ascending=False)
    
    ax = axes[idx]
    bars = ax.barh(subset['Model'], subset['F1-Score'], 
                   color=['#2ecc71' if i == 0 else '#3498db' for i in range(len(subset))])
    ax.set_xlabel('F1-Score', fontsize=11, fontweight='bold')
    ax.set_title(dataset_name, fontsize=13, fontweight='bold')
    ax.set_xlim([0, 1])
    ax.grid(axis='x', alpha=0.3)
    
    # Ajouter valeurs
    for i, bar in enumerate(bars):
        width = bar.get_width()
        ax.text(width, bar.get_y() + bar.get_height()/2, 
                f'{width:.3f}', 
                ha='left', va='center', fontsize=9, fontweight='bold')

plt.suptitle('Comparaison F1-Score par Dataset', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../results/visualizations/comparison_by_dataset_final.png', dpi=300, bbox_inches='tight')
print("✅ Sauvegardé: comparison_by_dataset_final.png")
plt.close()

# Tableau récapitulatif
summary = all_results.groupby('Dataset').agg({
    'Accuracy': 'max',
    'Precision': 'max',
    'Recall': 'max',
    'F1-Score': 'max',
    'ROC-AUC': 'max'
}).round(4)

print("\n📊 Résumé - Meilleure Performance par Dataset:")
print(summary)

# ============================================
# PARTIE 5: TEST SUR LE TEST SET - VÉRIFICATION OVERFITTING
# ============================================
print("\n" + "="*80)
print("🔬 ÉVALUATION SUR TEST SET - DÉTECTION OVERFITTING")
print("="*80)
print("⚠️  Le test set n'a JAMAIS été vu par les modèles!")
print("   → Performance réelle et honnête\n")

def evaluate_on_test_set(model_path, dataset_path, target_col, dataset_name):
    """Évalue un modèle sur le test set"""
    
    print(f"\n{'='*80}")
    print(f"📁 Dataset: {dataset_name}")
    print(f"{'='*80}")
    
    # Charger le modèle
    model = joblib.load(model_path)
    print(f"✅ Modèle chargé: {model_path.split('/')[-1]}")
    
    # Charger les données
    df = pd.read_csv(dataset_path)
    X = df.drop(target_col, axis=1)
    y = df[target_col]
    
    # Même split que l'entraînement (random_state=42)
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp)
    
    print(f"📊 Test set: {len(X_test)} samples")
    print(f"   Diabétiques: {y_test.sum()} ({y_test.sum()/len(y_test)*100:.1f}%)")
    
    # Prédictions
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Métriques
    test_metrics = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, zero_division=0),
        'Recall': recall_score(y_test, y_pred, zero_division=0),
        'F1-Score': f1_score(y_test, y_pred, zero_division=0),
        'ROC-AUC': roc_auc_score(y_test, y_proba) if y_proba is not None else None
    }
    
    print(f"\n🎯 RÉSULTATS TEST SET:")
    for metric, value in test_metrics.items():
        if value is not None:
            print(f"   {metric}: {value:.4f}")
    
    # Matrice de confusion
    cm = confusion_matrix(y_test, y_pred)
    print(f"\n📊 Matrice de Confusion:")
    print(cm)
    
    tn, fp, fn, tp = cm.ravel()
    print(f"\n   True Negatives:  {tn}")
    print(f"   False Positives: {fp}")
    print(f"   False Negatives: {fn}")
    print(f"   True Positives:  {tp}")
    
    # Classification report
    print(f"\n📋 Classification Report:")
    print(classification_report(y_test, y_pred, zero_division=0))
    
    return test_metrics, y_test, y_pred, y_proba, cm

# ============================================
# TEST 1: CLINICAL DATASET (XGBoost - 100% sur validation)
# ============================================
print("\n" + "🔍"*40)
print("TEST 1: CLINICAL DATASET - XGBoost (100% sur validation)")
print("🔍"*40)

clinical_metrics, clinical_y_test, clinical_y_pred, clinical_y_proba, clinical_cm = evaluate_on_test_set(
    model_path='../models/clinical/xgboost.pkl',
    dataset_path='../data/Dataset_10000_Lignes/dataset_clinical_ready.csv',
    target_col='Diabetes',
    dataset_name='Clinical (10000)'
)

# ============================================
# TEST 2: SYMPTOMS DATASET (Random Forest - 96.2%)
# ============================================
print("\n" + "🔍"*40)
print("TEST 2: SYMPTOMS DATASET - Random Forest (96.2% sur validation)")
print("🔍"*40)

symptoms_metrics, symptoms_y_test, symptoms_y_pred, symptoms_y_proba, symptoms_cm = evaluate_on_test_set(
    model_path='../models/symptoms/random_forest.pkl',
    dataset_path='../data/Dataset_600_Lignes/dataset_ready_for_ml.csv',
    target_col='class',
    dataset_name='Symptoms (600)'
)

# ============================================
# TEST 3: PIMA DATASET (Gradient Boosting - 68.3%)
# ============================================
print("\n" + "🔍"*40)
print("TEST 3: PIMA DATASET - Gradient Boosting (68.3% sur validation)")
print("🔍"*40)

pima_metrics, pima_y_test, pima_y_pred, pima_y_proba, pima_cm = evaluate_on_test_set(
    model_path='../models/pima/gradient_boosting.pkl',
    dataset_path='../data/Dataset_Pregnancies/pima_ready_for_ml.csv',
    target_col='Outcome',
    dataset_name='Pima (768)'
)

# ============================================
# PARTIE 6: COMPARAISON VALIDATION vs TEST
# ============================================
print("\n" + "="*80)
print("📊 COMPARAISON: VALIDATION SET vs TEST SET")
print("="*80)

comparison_df = pd.DataFrame({
    'Dataset': ['Clinical', 'Symptoms', 'Pima'],
    'Model': ['XGBoost', 'Random Forest', 'Gradient Boosting'],
    'Val_Accuracy': [
        results_clinical[results_clinical['Model'] == 'XGBoost']['Accuracy'].values[0],
        results_symptoms[results_symptoms['Model'] == 'Random Forest']['Accuracy'].values[0],
        results_pima[results_pima['Model'] == 'Gradient Boosting']['Accuracy'].values[0]
    ],
    'Test_Accuracy': [
        clinical_metrics['Accuracy'],
        symptoms_metrics['Accuracy'],
        pima_metrics['Accuracy']
    ],
    'Val_F1': [
        results_clinical[results_clinical['Model'] == 'XGBoost']['F1-Score'].values[0],
        results_symptoms[results_symptoms['Model'] == 'Random Forest']['F1-Score'].values[0],
        results_pima[results_pima['Model'] == 'Gradient Boosting']['F1-Score'].values[0]
    ],
    'Test_F1': [
        clinical_metrics['F1-Score'],
        symptoms_metrics['F1-Score'],
        pima_metrics['F1-Score']
    ]
})

# Calculer la différence (drop = overfitting)
comparison_df['Accuracy_Drop'] = comparison_df['Val_Accuracy'] - comparison_df['Test_Accuracy']
comparison_df['F1_Drop'] = comparison_df['Val_F1'] - comparison_df['Test_F1']

print("\n" + comparison_df.to_string(index=False))

# Sauvegarder
comparison_df.to_csv('../results/metrics/validation_vs_test_comparison.csv', index=False)
print("\n✅ Sauvegardé: validation_vs_test_comparison.csv")

# ============================================
# PARTIE 7: ANALYSE OVERFITTING
# ============================================
print("\n" + "="*80)
print("🔬 ANALYSE OVERFITTING")
print("="*80)

for idx, row in comparison_df.iterrows():
    dataset = row['Dataset']
    acc_drop = row['Accuracy_Drop'] * 100
    f1_drop = row['F1_Drop'] * 100
    
    print(f"\n📁 {dataset}:")
    print(f"   Accuracy drop: {acc_drop:.2f}%")
    print(f"   F1-Score drop: {f1_drop:.2f}%")
    
    if acc_drop < 1 and f1_drop < 1:
        print(f"   ✅ Excellent! Pas d'overfitting significatif")
    elif acc_drop < 5 and f1_drop < 5:
        print(f"   ⚠️  Léger overfitting (acceptable)")
    elif acc_drop < 10 and f1_drop < 10:
        print(f"   ⚠️  Overfitting modéré")
    else:
        print(f"   ❌ OVERFITTING SÉVÈRE!")

# ============================================
# PARTIE 8: VISUALISATION MATRICES DE CONFUSION
# ============================================
print("\n" + "="*80)
print("📊 CRÉATION MATRICES DE CONFUSION")
print("="*80)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

datasets_info = [
    ('Clinical', clinical_cm, 'XGBoost'),
    ('Symptoms', symptoms_cm, 'Random Forest'),
    ('Pima', pima_cm, 'Gradient Boosting')
]

for idx, (dataset_name, cm, model_name) in enumerate(datasets_info):
    ax = axes[idx]
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['Non-Diabétique', 'Diabétique'],
                yticklabels=['Non-Diabétique', 'Diabétique'],
                cbar=False)
    ax.set_title(f'{dataset_name}\n{model_name}', fontsize=12, fontweight='bold')
    ax.set_ylabel('Vraie Classe', fontsize=10)
    ax.set_xlabel('Classe Prédite', fontsize=10)

plt.suptitle('Matrices de Confusion - Test Set', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../results/visualizations/confusion_matrices_test.png', dpi=300, bbox_inches='tight')
print("✅ Sauvegardé: confusion_matrices_test.png")
plt.close()

# ============================================
# PARTIE 9: COURBES ROC
# ============================================
print("\n" + "="*80)
print("📈 CRÉATION COURBES ROC")
print("="*80)

plt.figure(figsize=(10, 8))

# Clinical
fpr_c, tpr_c, _ = roc_curve(clinical_y_test, clinical_y_proba)
roc_auc_c = auc(fpr_c, tpr_c)
plt.plot(fpr_c, tpr_c, label=f'Clinical - XGBoost (AUC = {roc_auc_c:.3f})', linewidth=2)

# Symptoms
fpr_s, tpr_s, _ = roc_curve(symptoms_y_test, symptoms_y_proba)
roc_auc_s = auc(fpr_s, tpr_s)
plt.plot(fpr_s, tpr_s, label=f'Symptoms - Random Forest (AUC = {roc_auc_s:.3f})', linewidth=2)

# Pima
fpr_p, tpr_p, _ = roc_curve(pima_y_test, pima_y_proba)
roc_auc_p = auc(fpr_p, tpr_p)
plt.plot(fpr_p, tpr_p, label=f'Pima - Gradient Boosting (AUC = {roc_auc_p:.3f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random (AUC = 0.5)', linewidth=1)
plt.xlabel('Taux de Faux Positifs', fontsize=12, fontweight='bold')
plt.ylabel('Taux de Vrais Positifs', fontsize=12, fontweight='bold')
plt.title('Courbes ROC - Test Set (Meilleurs Modèles)', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../results/visualizations/roc_curves_test.png', dpi=300, bbox_inches='tight')
print("✅ Sauvegardé: roc_curves_test.png")
plt.close()

# ============================================
# PARTIE 10: VERDICT FINAL
# ============================================
print("\n" + "="*80)
print("🏆 VERDICT FINAL")
print("="*80)

# Trouver le meilleur sur test set
best_test_f1 = max(clinical_metrics['F1-Score'], 
                   symptoms_metrics['F1-Score'], 
                   pima_metrics['F1-Score'])

if best_test_f1 == clinical_metrics['F1-Score']:
    winner = "Clinical Dataset - XGBoost"
    winner_metrics = clinical_metrics
elif best_test_f1 == symptoms_metrics['F1-Score']:
    winner = "Symptoms Dataset - Random Forest"
    winner_metrics = symptoms_metrics
else:
    winner = "Pima Dataset - Gradient Boosting"
    winner_metrics = pima_metrics

print(f"\n🥇 CHAMPION ABSOLU (Test Set): {winner}")
print(f"\n📊 Performance sur données JAMAIS VUES:")
print(f"   Accuracy:  {winner_metrics['Accuracy']:.4f} ({winner_metrics['Accuracy']*100:.2f}%)")
print(f"   Precision: {winner_metrics['Precision']:.4f} ({winner_metrics['Precision']*100:.2f}%)")
print(f"   Recall:    {winner_metrics['Recall']:.4f} ({winner_metrics['Recall']*100:.2f}%)")
print(f"   F1-Score:  {winner_metrics['F1-Score']:.4f} ({winner_metrics['F1-Score']*100:.2f}%)")
print(f"   ROC-AUC:   {winner_metrics['ROC-AUC']:.4f} ({winner_metrics['ROC-AUC']*100:.2f}%)")

# Recommandations
print("\n" + "="*80)
print("💡 RECOMMANDATIONS")
print("="*80)

if clinical_metrics['Accuracy'] > 0.95:
    print("\n✅ Le modèle Clinical-XGBoost est EXCELLENT!")
    print("   - Performance exceptionnelle sur données médicales")
    print("   - Prêt pour utilisation clinique (avec validation supplémentaire)")
    print("   - Dataset de 10,000 patients permet généralisation robuste")

if comparison_df['Accuracy_Drop'].max() > 0.1:
    print("\n⚠️  Attention: Certains modèles montrent de l'overfitting")
    print("   Solutions:")
    print("   - Augmenter regularisation")
    print("   - Réduire complexité du modèle")
    print("   - Collecter plus de données")

print("\n🎯 MODÈLE RECOMMANDÉ POUR PRODUCTION:")
print(f"   → {winner}")
print(f"   → F1-Score Test: {winner_metrics['F1-Score']:.4f}")

print("\n" + "="*80)
print("✅ ANALYSE COMPLÈTE TERMINÉE!")
print("="*80)
print("\n📁 Fichiers générés:")
print("   - heatmap_all_datasets.png")
print("   - comparison_by_dataset_final.png")
print("   - confusion_matrices_test.png")
print("   - roc_curves_test.png")
print("   - validation_vs_test_comparison.csv")
print("="*80)

📊 CHARGEMENT DES RÉSULTATS DE TOUS LES DATASETS

✅ Total: 23 modèles entraînés
   - Symptoms: 7 modèles
   - Clinical: 8 modèles
   - Pima: 8 modèles

🏆 MEILLEURS MODÈLES PAR DATASET (selon F1-Score)

📁 Symptoms (600):
   🥇 Modèle: Random Forest
   Accuracy:  0.8947
   Precision: 0.9231
   Recall:    0.9231
   F1-Score:  0.9231
   ROC-AUC:   0.9615

📁 Clinical (10000):
   🥇 Modèle: XGBoost
   Accuracy:  1.0000
   Precision: 1.0000
   Recall:    1.0000
   F1-Score:  1.0000
   ROC-AUC:   1.0000

📁 Pima (768):
   🥇 Modèle: Gradient Boosting
   Accuracy:  0.7739
   Precision: 0.6667
   Recall:    0.7000
   F1-Score:  0.6829
   ROC-AUC:   0.8367

🏆 CHAMPION ABSOLU (sur Validation Set)

🥇 Modèle: XGBoost
📁 Dataset: Clinical (10000)
📊 Métriques:
   Accuracy:  1.0000
   Precision: 1.0000
   Recall:    1.0000
   F1-Score:  1.0000
   ROC-AUC:   1.0000

⚠️  ATTENTION: 100% sur validation peut indiquer de l'overfitting!
   → On va TESTER sur le TEST SET pour vérifier...

📈 CRÉATION DES VISUALISATI